<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

import re
import urllib.request
import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs


### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [38]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Climate_change')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [39]:
citas = 357 #Numero de citas maximas
for i in range(citas):
  article_text = article_text.replace('['+str(i+1)+']', '')

In [40]:
article_text = article_text.replace('\n', '') #Remove end of line '\n'
article_text = re.split(r"\.\s*", article_text) #Split using period '.'

In [42]:
df = pd.DataFrame(article_text)

In [43]:
df.head()

,0
0,contemporary climate change includes both glob...
1,there have been previous periods of climate ch...
2,"instead, they are caused by the emission of gr..."
3,burning fossil fuels for energy use creates mo...
4,"certain agricultural practices, industrial pro..."


In [44]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 523


### 1 - Preprocesamiento

In [45]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [46]:
# Demos un vistazo
sentence_tokens[:2]

[['contemporary',
  'climate',
  'change',
  'includes',
  'both',
  'global',
  'warming',
  'and',
  'its',
  'impacts',
  'on',
  "earth's",
  'weather',
  'patterns'],
 ['there',
  'have',
  'been',
  'previous',
  'periods',
  'of',
  'climate',
  'change',
  'but',
  'the',
  'current',
  'changes',
  'are',
  'distinctly',
  'more',
  'rapid',
  'and',
  'not',
  'due',
  'to',
  'natural',
  'causes']]

### 2 - Crear los vectores (word2vec)

In [47]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [82]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=3,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [83]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [84]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 523


In [85]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 310


### 3 - Entrenar el modelo generador

In [86]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 93425.6796875
Loss after epoch 1: 43008.5546875
Loss after epoch 2: 41967.140625
Loss after epoch 3: 42388.03125
Loss after epoch 4: 42872.4375
Loss after epoch 5: 42536.25
Loss after epoch 6: 43060.1875
Loss after epoch 7: 41887.46875
Loss after epoch 8: 42469.75
Loss after epoch 9: 43115.09375
Loss after epoch 10: 42605.96875
Loss after epoch 11: 43555.25
Loss after epoch 12: 43217.25
Loss after epoch 13: 42524.75
Loss after epoch 14: 42529.5625
Loss after epoch 15: 42754.1875
Loss after epoch 16: 42660.25
Loss after epoch 17: 42939.6875
Loss after epoch 18: 42464.6875
Loss after epoch 19: 41585.25
Loss after epoch 20: 41015.0
Loss after epoch 21: 41183.5
Loss after epoch 22: 40259.6875
Loss after epoch 23: 39066.25
Loss after epoch 24: 35157.5
Loss after epoch 25: 35207.625
Loss after epoch 26: 35350.625
Loss after epoch 27: 35483.625
Loss after epoch 28: 35575.125
Loss after epoch 29: 34387.625
Loss after epoch 30: 34633.5
Loss after epoch 31: 34112.0
Loss after

(346606, 859800)

### 4 - Ensayar

In [75]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

KeyError: ignored

In [76]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["heat"], topn=10)

[('forestry', 0.060288697481155396),
 ('limiting', 0.024730538949370384),
 ('still', 0.015478290617465973),
 ('forcing', 0.015476329252123833),
 ('produce', 0.011324809864163399),
 ('emissions', 0.010515525005757809),
 ('adaptation', 0.008738206699490547),
 ('volcanic', 0.003406568430364132),
 ('scenario', 0.0008190330117940903),
 ('strategies', -0.003966944292187691)]

In [77]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["flooding"], topn=10)

[('coastal', 0.580795168876648),
 ('extreme', 0.5027619004249573),
 ('rise', 0.42892181873321533),
 ('increases', 0.404141902923584),
 ('level', 0.3769029378890991),
 ('sea', 0.3757351040840149),
 ('oxygen', 0.37245893478393555),
 ('space', 0.36624664068222046),
 ('protection', 0.36461061239242554),
 ('droughts', 0.3645362854003906)]

In [78]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["economic"], topn=5)

[('rapid', 0.4159148335456848),
 ('development', 0.38001173734664917),
 ('food', 0.3759843707084656),
 ('how', 0.35254615545272827),
 ('both', 0.32663241028785706)]

In [79]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["change"])

[('released', 0.02727360650897026),
 ('levels', 0.003059552051126957),
 ('vapour', 0.0028442665934562683),
 ('could', 0.0017433464527130127),
 ('than', -0.012157712131738663),
 ('increases', -0.028316307812929153),
 ('decade', -0.030176455155014992),
 ('reaching', -0.031862858682870865),
 ('reduction', -0.034022025763988495),
 ('radiation', -0.034187301993370056)]

### 5 - Visualizar agrupación de vectores

In [87]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [88]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=300
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

